## Practical 8 Image segmentation using Unet
## Submitted by: 18BCE088 Jyot Makadiya
### Objective: Implement Image segmentation on custom dataset.

Description: In this practical, we have implemented a UNet architecture for image segmentation task, we also have used generator for dataset. We have used VGG16 as a backbone pre-trained model. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp -r '/content/drive/MyDrive/Kaggle/dataset1/' -d './'

In [6]:
#importing libraries
import numpy as np
from tensorflow import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, UpSampling2D, concatenate, Lambda 
from keras.preprocessing.image import ImageDataGenerator

In [7]:
batch_size=2 
nepochs=5
nclasses=12

In [8]:
def Unet_with_vgg_encoder():
  #use pretrained weights from vgg16 with imagenet training, we can notice the input share also which is not square
  pretrained_model=VGG16(include_top=False,weights="imagenet",input_shape=(360,480,3))
  print(pretrained_model.summary())

  input_layer=pretrained_model.input
  output_layer_pretrained=pretrained_model.layers[-6].output
  encoder=Model(input_layer,output_layer_pretrained)
  #we set encoder false as we dont want to change their weights
  for layer in encoder.layers:
    layer.trainable=False
  print("Encoder summary:","-"*10)
  print(encoder.summary())

  lastpooling=MaxPooling2D(pool_size=(2,2),padding='same')(output_layer_pretrained)
  center=Conv2D(1024,(3,3),activation='relu',padding='same')(lastpooling)
  center=Conv2D(1024,(3,3),activation='relu',padding='same')(center)
  center=BatchNormalization()(center)

  up4=UpSampling2D((2,2))(center)
  up4=Conv2D(512,(2,2),activation='relu',padding='same')(up4)
  up4=Lambda(lambda x: x[:,0:45,:,:])(up4)
  up4=concatenate([up4,encoder.get_layer(name='block4_conv3').output],axis=3)
  up4=Conv2D(512,(3,3),activation='relu',padding='same')(up4)
  up4=Conv2D(512,(3,3),activation='relu',padding='same')(up4)
  up4=Conv2D(512,(3,3),activation='relu',padding='same')(up4)
  up4=BatchNormalization()(up4)

  up3=UpSampling2D((2,2))(up4)
  up3=Conv2D(256,(2,2),activation='relu',padding='same')(up3)
  up3=concatenate([up3,encoder.get_layer(name='block3_conv3').output],axis=3)
  up3=Conv2D(256,(3,3),activation='relu',padding='same')(up3)
  up3=Conv2D(256,(3,3),activation='relu',padding='same')(up3)
  up3=Conv2D(256,(3,3),activation='relu',padding='same')(up3)
  up3=BatchNormalization()(up3)

  up2=UpSampling2D((2,2))(up3)
  up2=Conv2D(128,(2,2),activation='relu',padding='same')(up2)
  up2=concatenate([up2,encoder.get_layer(name='block2_conv2').output],axis=3)
  up2=Conv2D(128,(3,3),activation='relu',padding='same')(up2)
  up2=Conv2D(128,(3,3),activation='relu',padding='same')(up2)
  up2=BatchNormalization()(up2)

  up1=UpSampling2D((2,2))(up2)
  up1=Conv2D(64,(2,2),activation='relu',padding='same')(up1)
  up1=concatenate([up1,encoder.get_layer(name='block1_conv2').output],axis=3)
  up1=Conv2D(64,(3,3),activation='relu',padding='same')(up1)
  up1=Conv2D(64,(3,3),activation='relu',padding='same')(up1)
  up1=Conv2D(nclasses,(1,1),activation='softmax',padding='same')(up1)

  model=Model(inputs=input_layer,outputs=up1)
  print(model.summary())
  return model

In [9]:
#defining the training generator for our task
def traingenerator():
  image_datagen=ImageDataGenerator(rescale=1/255)
  mask_datagen=ImageDataGenerator()

  image_batch_generator=image_datagen.flow_from_directory(
      directory="/content/dataset1/images_prepped_train",
      classes=[''],
      class_mode=None,target_size=(360,480),shuffle=False,seed=2021,batch_size=batch_size,color_mode='rgb'
  )
  
  mask_batch_generator=mask_datagen.flow_from_directory(
      directory="/content/dataset1/annotations_prepped_train",
      classes=[''],
      class_mode=None,target_size=(360,480),shuffle=False,seed=2021,batch_size=batch_size,color_mode='grayscale'
  )
  combine_generator=zip(image_batch_generator,mask_batch_generator)

  for i,j in combine_generator:
    new_mask=np.zeros(shape=(j.shape[0],j.shape[1],j.shape[2],nclasses))
    # print(new_mask.shape)
    for k in range(j.shape[0]):
      mask=j[k]
      # print(mask.shape)
      # break
      for m in range(nclasses):
        new_mask[k,:,:,m]=mask[:,:,0]==m
    #print(combine_generator.shape)
      # break
    yield(i,new_mask)

In [10]:
#defining the test generator
def test_generator():
  image_datagen_test=ImageDataGenerator(rescale=1/255)
  image_batch_generator_test=image_datagen_test.flow_from_directory(
      directory="/content/dataset1/images_prepped_test",
      classes=[''],
      class_mode=None,target_size=(360,480),shuffle=False,seed=2019,batch_size=batch_size,color_mode='rgb'
  )
  for i in image_batch_generator_test:
    yield i
    
unet=Unet_with_vgg_encoder()
unet.compile(optimizer=keras.optimizers.Adam(lr=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])
unet.fit(traingenerator(),steps_per_epoch=400,epochs=nepochs)
results=unet.predict(test_generator(),steps=51)

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 360, 480, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 360, 480, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 360, 480, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 180, 240, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 180, 240, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 180, 240, 128)     147584    
______________________________________________________________

We can notice the surprising result and accuracy from the model which is trained for not so long, this is due to starting from vgg16 network as an encoder which improved our model accuracy significantly.